<a href="https://colab.research.google.com/github/BhavyaParashar-codes/python_prompt/blob/main/study_planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()


Saving hesc107.xlsx to hesc107.xlsx


In [2]:
pip install pandas openpyxl

In [5]:
import pandas as pd
from datetime import datetime, timedelta

def generate_study_plan(file_path, total_days, daily_hours=3):
    try:
        df = pd.read_excel(file_path)
        df = df.dropna(subset=['Topic Title'])
        df['Content Length'] = df['Actual Content'].apply(lambda x: len(str(x)))

        def calculate_weight(row):
            weights = {
                'Paragraph': 1.0,
                'Image': 0.7,
                'Table': 0.8,
                'Activity': 1.5,
                'Question': 1.2,
                'Boxed Fact': 0.5
            }
            return weights.get(row['Type'], 1.0) * (1 + row['Content Length']/1000)

        df['Weight'] = df.apply(calculate_weight, axis=1)
        total_weight = df['Weight'].sum()
        weight_per_day = total_weight / total_days

        plan = []
        current_day = 1
        current_weight = 0
        day_topics = []
        start_date = datetime.now().date()

        for _, row in df.iterrows():
            if 'Exercise' in str(row['Topic Title']):
                continue

            if (current_weight + row['Weight'] > weight_per_day * 1.2) and day_topics:
                plan.append(create_day(
                    current_day,
                    start_date + timedelta(days=current_day-1),
                    day_topics,
                    'Study',
                    daily_hours
                ))
                current_day += 1
                current_weight = 0
                day_topics = []

                if current_day % 3 == 0 and current_day <= total_days:
                    plan.append(create_day(
                        current_day,
                        start_date + timedelta(days=current_day-1),
                        ['Review all previous topics'],
                        'Review',
                        daily_hours
                    ))
                    current_day += 1

            day_topics.append({
                'Chapter': row['Chapter Name'],
                'Topic': row['Topic Title'],
                'Type': row['Type'],
                'Content': str(row['Actual Content'])[:100] + '...',
                'Weight': row['Weight']
            })
            current_weight += row['Weight']

            if current_day > total_days:
                break

        if day_topics and current_day <= total_days:
            plan.append(create_day(
                current_day,
                start_date + timedelta(days=current_day-1),
                day_topics,
                'Study',
                daily_hours
            ))

        if current_day <= total_days:
            exercises = df[df['Topic Title'].str.contains('Exercise', na=False)]
            plan.append(create_day(
                total_days,
                start_date + timedelta(days=total_days-1),
                ['Final Assessment'] + exercises['Topic Title'].tolist(),
                'Assessment',
                daily_hours
            ))

        plan_df = pd.DataFrame(plan)
        output_file = f"Study_Plan_{total_days}days.xlsx"
        plan_df.to_excel(output_file, index=False)

        print(f"\nStudy plan generated successfully for {total_days} days!")
        print(f"Saved as: {output_file}")

        return plan_df

    except Exception as e:
        print(f"Error: {e}")
        return None

def create_day(day_num, date, topics, day_type, hours):
    activities = []
    resources = set(['Textbook'])

    for topic in topics:
        if isinstance(topic, str):
            activities.append(topic)
            continue

        if topic['Type'] == 'Paragraph':
            activities.append(f"Read: {topic['Topic']}")
        elif topic['Type'] == 'Image':
            activities.append(f"Study: {topic['Topic']}")
            resources.add('Visual Materials')
        elif topic['Type'] == 'Activity':
            activities.append(f"Do: {topic['Topic']}")
            resources.add('Lab Materials')
        elif topic['Type'] == 'Question':
            activities.append(f"Practice: {topic['Topic']}")
            resources.add('Question Bank')
        else:
            activities.append(f"Review: {topic['Topic']}")

    return {
        'Day': day_num,
        'Date': date,
        'Type': day_type,
        'Topics': ', '.join([t['Topic'] if isinstance(t, dict) else t for t in topics]),
        'Activities': '\n'.join(activities),
        'Resources': ', '.join(resources),
        'Duration': f"{hours} hours"
    }

if __name__ == "__main__":
    input_file = "hesc107.xlsx"
    study_days = 14
    print("Generating study plan...")
    plan = generate_study_plan(input_file, study_days)

    if plan is not None:
        print("\nHere's a preview of your plan:")
        print(plan[['Day', 'Date', 'Type', 'Topics']].head())

Generating study plan...

Study plan generated successfully for 14 days!
Saved as: Study_Plan_14days.xlsx

Here's a preview of your plan:
   Day        Date    Type                                             Topics
0    1  2025-07-28   Study  7.1 Adolescence and Puberty, 7.1 Adolescence a...
1    2  2025-07-29   Study  7.2 Changes at Puberty, 7.2 Changes at Puberty...
2    3  2025-07-30  Review                         Review all previous topics
3    4  2025-07-31   Study  7.2 Changes at Puberty, 7.2 Changes at Puberty...
4    5  2025-08-01   Study  7.2 Changes at Puberty, 7.2 Changes at Puberty...
